In [73]:
# Importing the required libraries
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

# Reading the dataset (Tennis-dataset)
data = pd.read_csv('Play_tennis.csv')

In [74]:
data.head()

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes


In [75]:
data.describe()

,day,outlook,temp,humidity,wind,play
count,14,14,14,14,14,14
unique,14,3,3,2,2,2
top,D1,Sunny,Mild,High,Weak,Yes
freq,1,5,6,7,8,9


In [76]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   day       14 non-null     object
 1   outlook   14 non-null     object
 2   temp      14 non-null     object
 3   humidity  14 non-null     object
 4   wind      14 non-null     object
 5   play      14 non-null     object
dtypes: object(6)
memory usage: 804.0+ bytes


In [77]:
data['play'].value_counts()

Yes    9
No     5
Name: play, dtype: int64

In [78]:
data['outlook'].value_counts()

Sunny       5
Rain        5
Overcast    4
Name: outlook, dtype: int64

In [79]:
data[data['outlook']=='Sunny']

,day,outlook,temp,humidity,wind,play
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
10,D11,Sunny,Mild,Normal,Strong,Yes


# checking Entropy and IG

In [80]:
import math
def calculate_entropy(df,target,feature='None',value='None'):
    value_array,loop_var = df[target].unique(),len(df[target].unique())
    Entropy = 0
    if feature == 'None':
        for i in range(0,loop_var):
            prob = len(df[df[target]==value_array[i]])/len(df)
            if prob != 0 :
                Entropy += (-prob * (math.log(prob,2)))
    else:
        df = df[df[feature]==value]
        for i in range (0,loop_var):
            val = len(df)
            if val == 0:
                val = 1
            prob = len(df[df[target]==value_array[i]])/val
            if prob != 0:
                    Entropy += (-prob * (math.log(prob,2)))
    return Entropy

In [81]:
calculate_entropy(data,'play')

0.9402859586706309

In [82]:
df=data.copy()

In [83]:
calculate_entropy(data,'play')

IG_Outlook = calculate_entropy(df,'play') -((len(df[df['outlook']=='Sunny'])/len(df))*calculate_entropy(df,'play','outlook','Sunny') 
 +(len(df[df['outlook']=='Overcast'])/len(df))*calculate_entropy(df,'play','outlook','Overcast') 
 +(len(df[df['outlook']=='Rain'])/len(df))*calculate_entropy(df,'play','outlook','Rain'))

IG_Outlook

0.2467498197744391

In [84]:
IG_Temp = calculate_entropy(df,'play') - ((len(df[df['temp']=='Hot'])/len(df))*calculate_entropy(df,'play','temp','Hot')
+ (len(df[df['temp']=='Mild'])/len(df))*calculate_entropy(df,'play','temp','Mild')
+(len(df[df['temp']=='Cool'])/len(df))*calculate_entropy(df,'play','temp','Cool'))
IG_Temp

0.029222565658954647

In [85]:
IG_Wind = calculate_entropy(df,'play') - ((len(df[df['wind']=='Weak'])/len(df))*calculate_entropy(df,'play','wind','Weak')
+ (len(df[df['wind']=='Strong'])/len(df))*calculate_entropy(df,'play','wind','Strong'))

IG_Wind

0.04812703040826927

In [86]:
IG_Humidity = calculate_entropy(df,'play') - ((len(df[df['humidity']=='High'])/len(df))*calculate_entropy(df,'play','humidity','High')
+ (len(df[df['humidity']=='Normal'])/len(df))*calculate_entropy(df,'play','humidity','Normal'))

IG_Humidity

0.15183550136234136

# Data Preparation

In [103]:
x=data.drop(columns='day')

In [104]:
x=data.drop(columns='play')
y=data['play']

In [105]:
from sklearn.preprocessing import LabelEncoder
Le = LabelEncoder()

data['outlook'] = Le.fit_transform(data['outlook'])
data['temp'] = Le.fit_transform(data['temp'])
data['humidity'] = Le.fit_transform(data['humidity'])
data['wind'] = Le.fit_transform(data['wind'])
data['play'] = Le.fit_transform(data['play'])


In [106]:
x.tail()

,day,outlook,temp,humidity,wind
9,D10,1,2,1,1
10,D11,2,2,1,0
11,D12,0,2,0,0
12,D13,0,1,1,1
13,D14,1,2,0,0


In [107]:
y.head()

0    0
1    0
2    1
3    1
4    1
Name: play, dtype: int64

# Build Model

In [108]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state=0)
model=DecisionTreeClassifier(criterion = 'entropy')
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
y_pred

ValueError: could not convert string to float: 'D14'

In [51]:
y_test

3     1
12    1
6     1
10    1
9     1
Name: play, dtype: int32

In [52]:
from sklearn.metrics import accuracy_score,classification_report

In [53]:
accuracy_score(y_test,y_pred)

NameError: name 'y_pred' is not defined

In [54]:
print(classification_report(y_test,y_pred))

NameError: name 'y_pred' is not defined

In [ ]:
#plot tree
from sklearn.tree import plot_tree
plot_tree(model)

In [21]:
from sklearn.tree import export_graphviz
dot_data=export_graphviz(model,feature_names=['outlook','temp','humidity','wind'],class_names=['Not Play','Play'],filled=True)

AttributeError: 'DecisionTreeClassifier' object has no attribute 'n_features_in_'

In [57]:
import graphviz

In [58]:
graphviz.Source(dot_data)

NameError: name 'dot_data' is not defined

# gini build model


In [59]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3)
model=DecisionTreeClassifier(criterion = 'gini')
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
y_pred

ValueError: could not convert string to float: 'D10'